### 数据来源
1、第三方数据接口<br>
    .http://tushare.org/trading.html#id3<br>
2、证券网站的交易明细数据<br>
    . http://quote.eastmoney.com/f1.html?code=600728&market=1<br>
    . http://vip.stock.finance.sina.com.cn/quotes_service/view/vMS_tradedetail.php?symbol=sz300044&date=2017-05-12&page=1

In [13]:
#coding=utf-8
import tushare as ts   #需要lxml等包的支持
import talib
import os,time,sys,re,datetime
import pandas as pd
import numpy as np

import datetime as dt

#### 网络数据库postgresql的使用

In [14]:
pro = ts.pro_api('6cbb7738f458c6366abd8601dc143cf493649fb97a7e6b4f43966902')
df = pro.daily(ts_code='600728.sh', start_date='20190401', end_date='20190416')
print(df)

      ts_code trade_date   open   high    low  close  pre_close  change  \
0   600728.SH   20190416  11.15  11.75  11.08  11.69      11.30    0.39   
1   600728.SH   20190415  11.59  11.82  11.26  11.30      11.30    0.00   
2   600728.SH   20190412  11.20  11.57  11.20  11.30      11.19    0.11   
3   600728.SH   20190411  11.70  11.77  11.12  11.19      11.59   -0.40   
4   600728.SH   20190410  11.77  11.77  11.24  11.59      11.76   -0.17   
5   600728.SH   20190409  11.84  12.15  11.66  11.76      11.84   -0.08   
6   600728.SH   20190408  12.26  12.37  11.58  11.84      12.38   -0.54   
7   600728.SH   20190404  12.75  12.92  12.35  12.38      12.78   -0.40   
8   600728.SH   20190403  12.68  13.06  12.61  12.78      12.85   -0.07   
9   600728.SH   20190402  12.72  13.45  12.60  12.85      12.65    0.20   
10  600728.SH   20190401  12.37  12.80  12.25  12.65      12.22    0.43   

    pct_chg         vol       amount  
0    3.4513   427840.42   492634.526  
1    0.0000   445302.

In [15]:
!pip install sqlalchemy

#### 桌面数据库access的使用
https://blog.csdn.net/growth_path_/article/details/80748734

In [16]:
import pyodbc  
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

[]

In [17]:
DBfile = r".\data\stock.mdb"  # 数据库文件  
try:
    conn = pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\data\stock.mdb')
    cur = conn.cursor()
except Exception as e:
     print('engine err!')

engine err!


In [18]:
#创建数据库表   可以在access中创建，也可用cur.execute('CREATE TABLE xxx()')来创建 

In [19]:
pro = ts.pro_api('6cbb7738f458c6366abd8601dc143cf493649fb97a7e6b4f43966902')


#### 自定义数据获取、处理与存储函数

In [20]:
#1、首先是获取沪深两市的股票列表
def Get_Stock_List():
    df = ts.get_stock_basics()
    df=df.sort_index() 
    #先只筛选某行业数据
    df=df[df['industry']=='电气设备']
    return df
          
#2）、获取1年交易数据，并保存
def Get_Stock_Trade(Code):
    df = None
    try:
            df = ts.get_hist_data(code, start='2018-01-01', end_date='2019-04-01',retry_count=5, pause=0.01)
            #df = pro.daily(Code, start_date='20190401', end_date='20190416')

    except Exception as e:
            print(code,'err',)
    return df

# 3)获取构造均价、均量、MACD数据
def Make_Full_stock(code):
    df =None
    try:
        df = Get_Stock_Trade(code)
    except Exception as e:
        print(code + 'trade err!')
        return df
 
    if not(df is None):
    
        # 构造成交价和成交量均线,根据具体情况构造n日均价，n日均线
        dfma25 = df['close'].rolling(min_periods=1, window=25, center=False).mean()   # 25日均价
        dfvol60 =df['volume'].rolling(min_periods=1, window=60, center=False).mean()  # 60日均量
 
      
        # 构造MACD值，psmacd DIFF；psmacdsignal DEA；psmacdhist DIFF-DEA
        # http://www.lfd.uci.edu/~gohlke/pythonlibs/ 下载安装 TA-Lib
        psmacd, psmacdsignal, psmacdhist = ta.MACD(np.array(df['close']), fastperiod=12, slowperiod=26, signalperiod=9)
        
        ##下面构造合并的df
        df['25']=dfma25
        
    return df

#4) 构造增加数据的Insert语句，以便将处理过某标的交易数据加入数据库
def Make_Sql(df,code):
    sqls = []
    print (df.iterrows())

    for tdate,row in df.iterrows():    #iter迭代的应用，要支持迭代
        #Open,high,low,close,volume,amount,,,,,, = row[1:n]
       
        sql = "insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,vol5) \
        values('%s','%s','%.2f','%.2f','%.2f','%.2f','%d','%.3f','%.3f','%.3f','%d')" \
        % (code, tdate, row['open'], row['high'], row['low'],row['close'],row['volume'],\
           row['ma5'],row['ma10'],row['ma20'],row['v_ma5'])
        
        sqls.append(sql)
 
    return sqls


# 5) 将处理过某标的交易数据加入数据库
def Insert_Stock_Trade(sqls, engine):
    try:
        for insert_sql in sqls:
            engine.execute(insert_sql)
            #print(insert_sql)
        engine.commit()
        print('Insert data over!')
    except Exception as e:
        print('sql err!')

#6) 构造各函数调用逻辑，完成程序功能
def Output_DB(df):
   #循环处理Get_Stock_List()获得的标的
    pass


In [21]:
Make_Full_stock('000049')

600628 err


#### 测试自定义函数

In [12]:
df = Get_Stock_List()    
df.head()

,name,industry,area,pe,outstanding,totals,totalAssets,liquidAssets,fixedAssets,reserved,...,bvps,pb,timeToMarket,undp,perundp,rev,profit,gpr,npr,holders
code,,,,,,,,,,,,,,,,,,,,,
000049,德赛电池,电气设备,深圳,17.46,2.05,2.05,942736.88,812170.00,67714.49,572.70,...,7.92,3.93,19950320,135080.63,6.58,50.19,47.38,8.16,2.35,26861.0
000400,许继电气,电气设备,河南,97.82,10.08,10.08,1431907.00,1202498.13,118774.85,83409.42,...,7.72,1.27,19970418,541913.81,5.37,15.39,-9.06,22.12,2.44,70680.0
000533,顺钠股份,电气设备,广东,0.00,6.85,6.91,377757.34,288739.00,22864.31,4081.43,...,0.93,3.25,19940103,72246.81,1.05,91.80,-191.36,2.57,-0.24,47638.0
000585,*ST东电,电气设备,海南,145.40,6.09,8.73,50511.34,18313.35,5013.68,107482.86,...,0.01,246.23,19951213,-201915.42,-2.31,-2.04,103.68,17.07,45.17,66700.0
000682,东方电子,电气设备,山东,41.61,9.78,13.41,515461.34,421455.69,58326.55,119931.77,...,2.25,2.36,19970121,45597.05,0.34,12.33,169.38,32.37,5.62,114800.0


In [12]:
code='600628'
df1=Get_Stock_Trade(code)
print(df1)
df1.head()

600628 err
None


AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
df1.shape


In [53]:
sqls=Make_Sql(df1,code)
print(sqls[:3])

<generator object DataFrame.iterrows at 0x11f70f2a0>
["insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,vol5)         values('600728','2019-04-23','11.16','11.28','10.98','11.14','256858','11.264','11.339','11.812','331192')", "insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,vol5)         values('600728','2019-04-22','11.28','11.46','11.11','11.16','353846','11.374','11.401','11.906','365389')", "insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,vol5)         values('600728','2019-04-19','11.18','11.30','11.05','11.26','270979','11.402','11.469','12.030','383680')"]


In [55]:
Insert_Stock_Trade(sqls, cur)

NameError: name 'cur' is not defined

###  复权数据获取
http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/000002.phtml?year=2017&jidu=1